## Compare liquid water paths

In [ ]:
import os
import netCDF4 as nc
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

Define paths where to find the data. Cloudnet data must be downloaded from Pangaea first!

In [ ]:
fname_tcwret = os.getenv('HOME') + '/code_richter_et_al/data_TCWret/mixed_ice_shape.nc'
path_cnet = os.getenv('HOME') + '/code_richter_et_al/get_cloudnet'

Read data from TCWret and save them as Pandas.DataFrame

In [ ]:
with nc.Dataset(fname_tcwret) as f:
    seconds = f.variables['time_of_measurement'][:]
    lwp = f.variables['liquid_water_path'][:]
    lwp_err = f.variables['liquid_water_path_error'][:]
    rliq = f.variables['liquid_water_effective_droplet_radius'][:]
    rice = f.variables['ice_water_effective_droplet_radius'][:]
    red_chi_2 = f.variables['reduced_chi_2'][:]
    t_cw = f.variables['liquid_water_optical_depth'][:] + f.variables['ice_water_optical_depth'][:]
    dof = f.variables['degrees_of_freedom_of_signal'][:]
    pwv = f.variables['precipitable_water_vapour'][:]

time = np.array([])
for ii in range(len(seconds)):
    sec = int(seconds[ii])
    time = np.concatenate((time, [dt.timedelta(seconds=sec) + dt.datetime(2017, 5, 1)]))
    
tcwret_raw = pd.DataFrame({'time': time, 'pwv(cm)': pwv, 'rliq(um)': rliq, 'rice(um)': rice, 'lwp(gm-2)': lwp, 'dlwp(gm-2)': lwp_err, 'red_chi_2(1)': red_chi_2, 'tcw(1)': t_cw, 'dof(1)': dof})

Apply filtering of data

In [ ]:
tau_max = 6.0
tau_min = 0.0

idx_conv = np.where((tcwret_raw['red_chi_2(1)'] <= 1.0) & (tcwret_raw['red_chi_2(1)'] >= 0.0))[0]
idx_tau = np.where((tcwret_raw['tcw(1)'] <= tau_max) & (tcwret_raw['tcw(1)'] >= tau_min))[0]
idx_valid = np.intersect1d(idx_conv, idx_tau)
tcwret = tcwret_raw.iloc[idx_valid]

counter = 0
idx = np.array([])
for ii in range(len(tcwret)):
    if tcwret['rliq(um)'].iloc[ii] < tcwret['rice(um)'].iloc[ii]:
        idx = np.concatenate((idx, [ii]))
        counter += 1
tcwret = tcwret.iloc[idx]

Read Cloudnet data. Only allow retrieval flags 0,1,2 and 3

In [ ]:
lwc_st_invalid = [4,5,6]
cloudnet = {'time': [], 'lwp(gm-2)': [], 'lwp_mwr(gm-2)': [], 'lwp_err_mwr(gm-2)': []}
for file_ in sorted(os.listdir(path_cnet)):
    if ".nc" in file_:
        with nc.Dataset(os.path.join(path_cnet, file_)) as f:
            if "cnet" not in file_:
                continue
            day_month = dt.datetime.strptime(file_, 'cnet_%m_%d.nc')
            day = dt.datetime(2017, day_month.month, day_month.day)
            time = f.variables['datetime'][:]
            lwp = f.variables['liquid_water_path_per_layer'][:]
            lwp_st = f.variables['liquid_water_content_status'][:]
            iwp_st = f.variables['ice_water_content_status'][:]
            lwp_mwr = f.variables['liquid_water_path_MWR'][:]
            lwp_err_mwr = f.variables['liquid_water_path_error_MWR'][:]
            for time_idx in range(len(time)):
                time_iter = day + dt.timedelta(seconds=int(np.round(time[time_idx]*3600)))
                idx_liq = np.where(lwp[time_idx] > 0.0)[0]               

                lwc_invalid = np.intersect1d(lwp_st[time_idx], np.array(lwc_st_invalid))
                if lwc_invalid.size != 0 or iwc_invalid.size != 0: continue

                cloudnet['time'].append(time_iter)
                cloudnet['lwp_mwr(gm-2)'].append(lwp_mwr[time_idx])
                cloudnet['lwp_err_mwr(gm-2)'].append(lwp_err_mwr[time_idx])
                cloudnet['lwp(gm-2)'].append(np.sum(lwp[time_idx]))
cloudnet = pd.DataFrame(cloudnet)
idx = np.array([])
for ii in range(len(cloudnet)):
    if not np.ma.is_masked(cloudnet['lwp_mwr(gm-2)'].iloc[ii]) and not np.ma.is_masked(cloudnet['lwp(gm-2)'].iloc[ii]):
        idx = np.concatenate((idx, [ii]))
idx = np.array(idx, dtype=int)
cloudnet = cloudnet.iloc[idx]

Define averaging time interval

In [ ]:
delta = 2

Average Cloudnet data

In [ ]:
cloudnet['time'].iloc[0].to_pydatetime()
lwp_mean = []
lwp_mean_mwr = []
iwp_mean = []
cwp_mean = []
rliq_mean = []
rice_mean = []
lat_mean = []
lon_mean = []
time_mean = []
lwp_mean_err_mwr = []
dp = []
datetime_start = np.datetime64("2017-05-24T20:25:00")
datetime_iter = datetime_start
datetime_stop = np.datetime64("2017-07-18T00:00:00")
while datetime_iter < datetime_stop:
    idx = np.where((np.array(cloudnet['time']) > datetime_iter) & \
                   (np.array(cloudnet['time']) < datetime_iter+np.timedelta64(delta*60, 's')))[0]
    if idx.size != 0:
        lwp_mean_mwr.append(np.mean(np.array(cloudnet['lwp_mwr(gm-2)'])[idx]))
        lwp_mean_err_mwr.append(np.mean(np.array(cloudnet['lwp_err_mwr(gm-2)'])[idx]))
        lwp_mean.append(np.mean(np.array(cloudnet['lwp(gm-2)'])[idx]))
        time_mean.append(datetime_iter)
        dp.append(idx.size)
    datetime_iter += np.timedelta64(delta*60, 's')
    
cloudnet_av = pd.DataFrame({'time': time_mean, 'lwp_mwr(gm-2)': lwp_mean_mwr, 'lwp(gm-2)': lwp_mean, 'lwp_err_mwr(gm-2)': lwp_mean_err_mwr, 'dp': dp})

Average TCWret data

In [ ]:
lwp_mean = []
dlwp_mean = []
iwp_mean = []
cwp_mean = []
dof_mean = []
rliq_mean = []
rice_mean = []
lat_mean = []
lon_mean = []
time_mean = []
tcw_mean = []
chi_mean = []
pwv_mean = []
datetime_start = np.datetime64("2017-05-24T20:25:00")
datetime_iter = datetime_start
datetime_stop = np.datetime64("2017-07-18T00:00:00")
while datetime_iter < datetime_stop:
    idx = np.where((np.array(tcwret['time']) > datetime_iter) & \
                   (np.array(tcwret['time']) < datetime_iter+np.timedelta64(delta*60, 's')))[0]
    if idx.size != 0:
        tcw_mean.append(np.mean(np.array(tcwret['tcw(1)'])[idx]))
        lwp_mean.append(np.mean(np.array(tcwret['lwp(gm-2)'])[idx]))
        dlwp_mean.append(np.mean(np.array(tcwret['dlwp(gm-2)'])[idx]))
        dof_mean.append(np.mean(np.array(tcwret['dof(1)'])[idx]))
        chi_mean.append(np.mean(np.array(tcwret['red_chi_2(1)'])[idx]))
        pwv_mean.append(np.mean(np.array(tcwret['pwv(cm)'])[idx]))
        time_mean.append(datetime_iter)
    datetime_iter += np.timedelta64(delta*60, 's')
    
tcwret_av = pd.DataFrame({'time': time_mean, 'lwp(gm-2)': lwp_mean, 'dlwp(gm-2)': dlwp_mean, 'tcw(1)': tcw_mean, 'dof(1)': dof_mean, 'red_chi_2(1)': chi_mean, 'pwv(cm)': pwv_mean})

Calculate correlation coefficient, p-Value, mean and standard deviation

In [ ]:
intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
xax = np.array(tcwret_av['lwp(gm-2)'].iloc[idx_tcwret])
yax = np.array(cloudnet_av['lwp_mwr(gm-2)'].iloc[idx_cloudnet])
print("Data\t\t\tcor\tp-Value\tMean\tSD\tNumber")
pearsonr, pval = scipy.stats.pearsonr(xax,yax)
print("LWP All\t\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))

intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
xax = np.array(tcwret_av['lwp(gm-2)'].iloc[idx_tcwret])
yax = np.array(cloudnet_av['lwp_mwr(gm-2)'].iloc[idx_cloudnet])
cax = np.array(tcwret_av['pwv(cm)'].iloc[idx_tcwret])
idx = np.where((cax <= 1.0))[0]
xax = xax[idx]
yax = yax[idx]
pearsonr, pval = scipy.stats.pearsonr(xax,yax)
print("LWP PWV < 1.0\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))

intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
xax = np.array(tcwret_av['lwp(gm-2)'].iloc[idx_tcwret])
yax = np.array(cloudnet_av['lwp_mwr(gm-2)'].iloc[idx_cloudnet])
cax = np.array(tcwret_av['pwv(cm)'].iloc[idx_tcwret])
idx = np.where(yax <= 20.0)[0]
xax = xax[idx]
yax = yax[idx]
pearsonr, pval = scipy.stats.pearsonr(xax,yax)
print("LWP < 20.0\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))

intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
xax = np.array(tcwret_av['lwp(gm-2)'].iloc[idx_tcwret])
yax = np.array(cloudnet_av['lwp_mwr(gm-2)'].iloc[idx_cloudnet])
cax = np.array(tcwret_av['pwv(cm)'].iloc[idx_tcwret])
idx = np.where((yax <= 20.0) & (cax <= 1.0))[0]
xax = xax[idx]
yax = yax[idx]
pearsonr, pval = scipy.stats.pearsonr(xax,yax)
print("LWP < 20.0 PWV < 1.0\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))